# db에서 불러오기

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
# from sqlalchemy import create_engine

In [ ]:
# 1. 데이터베이스 연결 -> 연결 객체 반환
conn = pymysql.connect(host="localhost", port=3306, db="mp",
                       user="root", passwd="1234", cursorclass=pymysql.cursors.DictCursor)

cursor = conn.cursor()

In [ ]:
# 3. SQL 작성 + 명령 실행 ( 조회 명령 )
sql = """select * from fine_dust"""
cursor.execute(sql)
result = cursor.fetchall()
result

In [ ]:
fine_dust_df = pd.DataFrame(result)
fine_dust_df.head()

In [ ]:
conn.close()
cursor.close()

### DB에서 불러온 데이터를 csv파일로 저장

In [ ]:
fine_dust_df.to_csv('./data/fine_dust.csv')

# db 종료 후 DF 작업하기 (여기서부터 시작)

In [26]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
# from sqlalchemy import create_engine
# !pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew, kurtosis
import os

In [2]:
# matplotlib 한글 사용

plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

### 데이터 불러와서 가공하기

In [3]:
fine_dust_df = pd.read_csv('./data/fine_dust.csv')
fine_dust_df.columns

Index(['Unnamed: 0', 'station_id', 'station_name', 'date', 'fine_dust(㎍/㎥)'], dtype='object')

In [4]:
# 필요 컬럼만 자르기
fine_dust = fine_dust_df.loc[:, ['date', 'fine_dust(㎍/㎥)']] # 잘랐으면 비활성화 안 잘랐으면 활성화
fine_dust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            1323 non-null   object
 1   fine_dust(㎍/㎥)  1323 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.8+ KB


미세먼지 등급별 값 메기기

In [5]:
# 좋음 (0-30 ㎍/m³) → Good
# 보통 (31-80 ㎍/m³) → Moderate
# 나쁨 (81-150 ㎍/m³) → Bad
# 매우 나쁨 (151 ㎍/m³ 이상) → Very Bad

# 새로운 컬럼을 생성하고 조건에 따라 값 입력

conditionlist = [
    (fine_dust['fine_dust(㎍/㎥)'] <= 30),
    (fine_dust['fine_dust(㎍/㎥)'] > 30) & (fine_dust['fine_dust(㎍/㎥)'] <= 80),
    (fine_dust['fine_dust(㎍/㎥)'] > 80) & (fine_dust['fine_dust(㎍/㎥)'] <= 150),
    (fine_dust['fine_dust(㎍/㎥)'] > 150),
]

values = ['good', 'normal', 'bad', 'very_bad']

fine_dust["grade"] = np.select(conditionlist, values, default="unknown")
fine_dust.head(2)

,date,fine_dust(㎍/㎥),grade
0,2021-01-01,36,normal
1,2021-01-02,43,normal


fine_dust의 date의 타입 변경 후 평일/주말 나누기

In [6]:
fine_dust.info() # date 타입이 object임을 확인
fine_dust['date'] = pd.to_datetime(fine_dust['date'])
# fine_dust.info() # date타입이 datetime64[ns]으로 변경됨

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            1323 non-null   object
 1   fine_dust(㎍/㎥)  1323 non-null   int64 
 2   grade           1323 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.1+ KB


In [7]:
# fine_dust["date"]에 따라 평일, 주말 나누기(월~금 : 0~4, 토일 : 5, 6)
fine_dust["day"] = np.where(fine_dust["date"].dt.weekday < 5, "weekday", "weekend")
fine_dust


,date,fine_dust(㎍/㎥),grade,day
0,2021-01-01,36,normal,weekday
1,2021-01-02,43,normal,weekend
2,2021-01-03,47,normal,weekend
3,2021-01-04,50,normal,weekday
4,2021-01-05,37,normal,weekday
...,...,...,...,...
1318,2024-10-25,29,good,weekday
1319,2024-10-26,26,good,weekend
1320,2024-10-27,27,good,weekend
1321,2024-10-28,38,normal,weekday


공휴일 데이터인 hd의 date의 타입 변경

In [8]:
hd = pd.read_csv('./data/hd_21_24.csv', encoding="CP949")
hd.rename(columns={"일시":"date"}, inplace=True)
hd['date'] = pd.to_datetime(hd['date'])
hd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      66 non-null     datetime64[ns]
 1   dateName  66 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 1.2+ KB


fine_dust['day_type'] 만들어서 평일/주말+공휴일로 나누기 (공휴일은 주말로 처리)

In [9]:
# fine_dust와 hd DF 합치기
m_fine_dust = pd.merge(fine_dust, hd, how='outer', on='date')
m_fine_dust.head()

,date,fine_dust(㎍/㎥),grade,day,dateName
0,2021-01-01,36.0,normal,weekday,1월1일
1,2021-01-02,43.0,normal,weekend,NaN
2,2021-01-03,47.0,normal,weekend,NaN
3,2021-01-04,50.0,normal,weekday,NaN
4,2021-01-05,37.0,normal,weekday,NaN


In [10]:
# apply 활용
date_name2 = m_fine_dust.apply(lambda row : 'weekend' 
                               if str(row['dateName']) != 'nan' else row['day'], axis=1)

In [11]:
# apply 활용해서 평일, 주말(+공휴일)로 나눈 값을
# 합친 df에 적용하기 
m_fine_dust['day_type'] = date_name2

# 필요한 컬럼/행으로 자르기
m_fine_dust = m_fine_dust.loc[:, ['date', 'fine_dust(㎍/㎥)', 'grade', 'day_type']]
m_fine_dust = m_fine_dust.iloc[:1329, :]
m_fine_dust.head(1), m_fine_dust.tail(1)

(        date  fine_dust(㎍/㎥)   grade day_type
 0 2021-01-01            36.0  normal  weekend,
            date  fine_dust(㎍/㎥) grade day_type
 1328 2024-10-30            28.0  good  weekday)

병합된 DF을 계절별, 연도별로 구분하기

In [12]:
# date를 기준으로 계절별 구분 (봄: 3~5, 여름: 6~8, 가을: 9~11, 겨울: 12~2)
conditions = [
    ( m_fine_dust["date"].dt.month >= 3 ) & ( m_fine_dust["date"].dt.month <= 5 ),
    ( m_fine_dust["date"].dt.month >= 6 ) & ( m_fine_dust["date"].dt.month <= 8 ),
    ( m_fine_dust["date"].dt.month >= 9 ) & ( m_fine_dust["date"].dt.month <= 11 ),
]
season = ["Spring", "Summer", "Fall"]
m_fine_dust["season"] = np.select(conditions, season, default="Winter")

print( m_fine_dust.iloc[57:59, :] )
print( m_fine_dust.iloc[145:147, :] )
print( m_fine_dust.iloc[237:239, :] )
print( m_fine_dust.iloc[325:327, :] )

         date  fine_dust(㎍/㎥)   grade day_type  season
57 2021-02-28            32.0  normal  weekend  Winter
58 2021-03-01             NaN     NaN  weekend  Spring
          date  fine_dust(㎍/㎥)   grade day_type  season
145 2021-05-31            59.0  normal  weekday  Spring
146 2021-06-01            52.0  normal  weekday  Summer
          date  fine_dust(㎍/㎥)   grade day_type  season
237 2021-08-31            37.0  normal  weekday  Summer
238 2021-09-01             9.0    good  weekday    Fall
          date  fine_dust(㎍/㎥)   grade day_type  season
325 2021-11-30            31.0  normal  weekday    Fall
326 2021-12-01            19.0    good  weekday  Winter


In [13]:
# date를 기준으로 연도 구분
m_fine_dust["year"] = m_fine_dust["date"].dt.year
print( m_fine_dust.iloc[356:358, :] )

          date  fine_dust(㎍/㎥)   grade day_type  season  year
356 2021-12-31            21.0    good  weekday  Winter  2021
357 2022-01-01            34.0  normal  weekend  Winter  2022


결측치 수정 ( 미세먼지 농도 결측치를 포함되는 계절의 평균으로 대체 )

In [14]:
print( "평일 수 : ", len(m_fine_dust.loc[m_fine_dust["day_type"] == "weekday"]) )
print( "주말 및 공휴일 수 : ", len(m_fine_dust.loc[m_fine_dust["day_type"] == "weekend"]) )

평일 수 :  902
주말 및 공휴일 수 :  427


In [15]:
# m_fine_dust[m_fine_dust['fine_dust(㎍/㎥)'].isnull()]
# 결측치 행
        # 58	2021-03-01	NaN	NaN	weekend
        # 504	2022-06-06	NaN	NaN	weekend
        # 620	2022-10-10	NaN	NaN	weekend
        # 822	2023-05-05	NaN	NaN	weekend
        # 1176	2024-05-15	NaN	NaN	weekend
        # 1293	2024-09-16	NaN	NaN	weekend
# 계절별 평균을 결측치에 넣기로 함

# 계절별 평균 구하기
season_means = m_fine_dust.groupby('season')['fine_dust(㎍/㎥)'].mean()
                # Fall      28.840909
                # Spring    57.397101
                # Summer    28.062500
                # Winter    46.641509

# 결측치행에 평균값 넣기
m_fine_dust["fine_dust(㎍/㎥)"] = m_fine_dust["fine_dust(㎍/㎥)"]. \
                        fillna(m_fine_dust.groupby("season")["fine_dust(㎍/㎥)"].transform("mean"))

m_fine_dust[m_fine_dust['fine_dust(㎍/㎥)'].isnull()]


,date,fine_dust(㎍/㎥),grade,day_type,season,year


범주화

In [16]:
# 범주화
# 순서있는 범주형 : ordered = True, 순서 없는 범주형 : ordered = False
c_grade = ['good', 'normal', 'bad', 'very_bad']
c_grade1 = CategoricalDtype(categories = c_grade, ordered = True)

c_season = ["Spring", "Summer", "Fall", "Winter"]
c_season1 = CategoricalDtype(categories = c_season, ordered = True)

m_fine_dust["grade"] = m_fine_dust["grade"].astype(c_grade1)
m_fine_dust["season"] = m_fine_dust["season"].astype(c_season1)
m_fine_dust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329 entries, 0 to 1328
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1329 non-null   datetime64[ns]
 1   fine_dust(㎍/㎥)  1329 non-null   float64       
 2   grade           1323 non-null   category      
 3   day_type        1329 non-null   object        
 4   season          1329 non-null   category      
 5   year            1329 non-null   int32         
dtypes: category(2), datetime64[ns](1), float64(1), int32(1), object(1)
memory usage: 39.5+ KB


### 데이터 분석

1. 단일 그룹 분석

In [ ]:
def group_analysis_by_single_column(df, group_col_name, col_name):
    # 그룹화
    grouped_df = df.groupby(group_col_name, observed=True)

    # 그룹별 최대/최소/평균/분산/빈도 계산
    stats = grouped_df[col_name].agg([
        ("Max", "max"),
        ("Min", "min"),
        ("Mean", "mean"),
        ("Variance", "var"),
        ("Count", "count")
    ])
    return stats

# 단일 그룹 분석 결과를 데이터 프레임으로 저장
df_year_stats = group_analysis_by_single_column(
                    df = m_fine_dust, 
                    group_col_name = "year", 
                    col_name = "fine_dust(㎍/㎥)")
df_grade_stats = group_analysis_by_single_column(
                    df = m_fine_dust, 
                    group_col_name = "grade", 
                    col_name = "fine_dust(㎍/㎥)")
df_season_stats = group_analysis_by_single_column(
                    df = m_fine_dust, 
                    group_col_name = "season", 
                    col_name = "fine_dust(㎍/㎥)")
df_day_stats = group_analysis_by_single_column(
                    df = m_fine_dust, 
                    group_col_name = "day_type", 
                    col_name = "fine_dust(㎍/㎥)")

# 딕셔너리로 데이터프레입 합치기
combined_stats = {
    "Year": df_year_stats,
    "Grade": df_grade_stats,
    "Season": df_season_stats,
    "Day Type": df_day_stats
}

print( "단일 그룹 분석")
# 결과 확인
for key, value in combined_stats.items():
    print(f"\n\n{key} Stats:\n", value)


단일 그룹 분석


Year Stats:
         Max   Min       Mean     Variance  Count
year                                            
2021  389.0   9.0  48.090748  1654.192038    357
2022  247.0   8.0  36.072394   457.202489    344
2023  333.0  10.0  41.469532   933.951426    348
2024  277.0  10.0  34.443707   587.494320    280


Grade Stats:
             Max    Min        Mean     Variance  Count
grade                                                 
good       30.0    8.0   22.112436    28.751842    587
normal     79.0   31.0   45.910198   152.853204    657
bad       148.0   81.0  106.938462   263.902404     65
very_bad  389.0  152.0  235.785714  7523.104396     14


Season Stats:
           Max   Min       Mean     Variance  Count
season                                            
Spring  389.0  11.0  57.397101  2095.632845    348
Summer   72.0   8.0  28.062500   121.035866    353
Fall    140.0   9.0  28.840909   216.638202    310
Winter  247.0  11.0  46.641509   715.416820    318


Day Type St

- 단일 그룹 분석 시각화

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_stats_with_count(stats_df, ax, title):
    """
    Parameters:
        stats_df: 통계치를 포함한 데이터 프레임
        ax: 서브플롯의 축
        title: 그래프 제목
    """
    # 막대 그래프 생성
    bars = stats_df[['max', 'min', 'mean']].plot(kind='bar', ax=ax, 
                                                color=['skyblue', 'salmon', 'lightgreen'],
                                                width=0.8)
    
    # 2024년 막대에 패턴 추가
    if 'year' in str(stats_df.index.name).lower():  # year 관련 그래프인 경우에만
        for container in bars.containers:  # max, min, mean 각각의 막대에 대해
            # 2024년에 해당하는 막대에 패턴 추가
            for idx, patch in enumerate(container):
                if str(stats_df.index[idx]) == '2024':
                    patch.set_hatch('///')
                    patch.set_alpha(0.8)
    
    # 두 번째 y축 생성
    ax2 = ax.twinx()
    
    # count를 선 그래프로 표시
    count_line = ax2.plot(range(len(stats_df)), stats_df['count'], 
                         color='sandybrown', marker='o', linewidth=2, 
                         label='count')
    
    # 첫 번째 축 설정
    ax.set_title(title)
    ax.set_xlabel(stats_df.index.name)
    ax.set_ylabel('Values')
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)
    
    # 두 번째 축 설정
    ax2.set_ylabel('Count')
    ax2.tick_params(axis='y', labelcolor='sandybrown')
    
    # 범례 통합
    lines1, labels1 = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines1 + count_line, labels1 + ['count'], 
              loc='upper right')
    
    # 2024년 데이터가 부분적임을 설명하는 주석 추가
    if 'year' in str(stats_df.index.name).lower():
        ax.annotate('* 2024 data: Jan-Oct 30th', 
                   xy=(0.02, 0.98), 
                   xycoords='axes fraction',
                   bbox=dict(facecolor='white', edgecolor='gray', alpha=0.8),
                   fontsize=8,
                   va='top')

# 서브플롯 생성
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

# 각 서브플롯에 그래프 그리기
plot_stats_with_count(df_year_stats, axs[0, 0], 
                     'Yearly Fine Dust Statistics')
plot_stats_with_count(df_grade_stats, axs[0, 1], 
                     'Grade-wise Fine Dust Statistics')
plot_stats_with_count(df_season_stats, axs[1, 0], 
                     'Seasonal Fine Dust Statistics')
plot_stats_with_count(df_day_stats, axs[1, 1], 
                     'Weekday/Weekend Fine Dust Statistics')

# 레이아웃 조정
plt.tight_layout()
plt.show()

In [ ]:
# 그래프를 파일로 저장
plt.savefig('./data/Single_Group_Analysis_Plots.png', 
            dpi=300,  # 해상도 설정
            bbox_inches='tight',  # 여백 자동 조정
            pad_inches=0.1)  # 여백 크기

그래프별 데이터 분석 (분석내용 주석처리)

<!-- 
Yearly Fine Dust Statistics (연도별 미세먼지 통계):
2021년이 가장 높은 최대값(약 380)
2022-2023년은 최대값이 감소하는 추세
2024년(1월-10월30일)은 다른 연도보다 데이터 수(count)가 적음
전반적으로 평균값은 50 미만으로 유지됨

Grade-wise Fine Dust Statistics (등급별 미세먼지 통계):
very_bad(매우 나쁨) 등급이 가장 높은 최대값(약 380)
등급이 나빠질수록(good→very_bad) 최대값, 최소값, 평균값 모두 증가
good(좋음)과 normal(보통)은 상대적으로 많은 발생 빈도
very_bad는 발생 빈도가 가장 낮음

Seasonal Fine Dust Statistics (계절별 미세먼지 통계):
봄(Spring)에 가장 높은 최대값
여름(Summer)과 가을(Fall)은 상대적으로 낮은 수치
봄과 여름이 다른 계절보다 발생 빈도가 높음
겨울(Winter)은 중간 정도의 수치를 보임

Weekday/Weekend Fine Dust Statistics (평일/주말 미세먼지 통계):
평일(weekday)과 주말(weekend)의 최대값이 비슷
평일이 주말보다 약간 더 많은 발생 빈도
평균값은 평일과 주말이 비슷한 수준(약 40)

전반적인 특징:
계절성: 봄철에 미세먼지가 가장 심각
연도별 개선: 2021년 이후 최대값이 감소하는 추세
발생 빈도: 좋음/보통 등급이 가장 많이 발생
평일/주말 차이: 큰 차이는 없으나 평일이 조금 더 빈번

미세먼지 관리나 대책 수립 시 봄철에 더 집중적인 관리가 필요하며, 
전반적으로 미세먼지 상황이 개선되고 있다는 것을 알 수 있음
 -->

2. 두 개 변수 조합 분석

In [ ]:
# 2-1. 연도와 등급을 조합해서 분석
# 2-2. 연도와 계절을 조합해서 분석
# 2-3. 연도와 평일/주말을 조합해서 분석
# 2-4. 등급과 계절을 조합해서 분석
# 2-5. 등급과 평일/주말을 조합해서 분석
# 2-6. 계절과 평일/주말을 조합해서 분석

In [18]:
def group_analysis_by_two_columns(df, group_col1, group_col2, target_col):
    # 두 컬럼으로 그룹화
    grouped_df = df.groupby([group_col1, group_col2], observed=True)
    
    # 통계 계산 및 컬럼 이름 지정 (열 이름, 계산 방법)
    stats = grouped_df[target_col].agg([
    ("Max", "max"),
    ("Min", "min"),
    ("Mean", "mean"),
    ("Variance", "var"),
    ("Count", "count"),
    ("Median", "median"),                   # 중앙값
    ("Q1", lambda x: x.quantile(0.25)),     # 1사분위수 (Q1)
    ("Q3", lambda x: x.quantile(0.75)),     # 3사분위수 (Q3)
    ("IQR", lambda x: x.quantile(0.75) - x.quantile(0.25)),     # IQR
    ("Skewness", lambda x: skew(x)),        # 왜도
    ("Kurtosis", lambda x: kurtosis(x)),    # 첨도
    ("Range", lambda x: x.max() - x.min())  # 범위
])
    return stats

# 함수 실행 예시
df_year_grade_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "year", 
    group_col2 = "grade", 
    target_col = "fine_dust(㎍/㎥)")
df_year_season_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "year", 
    group_col2 = "season", 
    target_col = "fine_dust(㎍/㎥)")
df_year_day_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "year", 
    group_col2 = "day_type", 
    target_col = "fine_dust(㎍/㎥)")
df_grade_season_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "grade", 
    group_col2 = "season", 
    target_col = "fine_dust(㎍/㎥)")
df_grade_day_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "grade", 
    group_col2 = "day_type", 
    target_col = "fine_dust(㎍/㎥)")
df_season_day_stats = group_analysis_by_two_columns(
    df = m_fine_dust,
    group_col1 = "season", 
    group_col2 = "day_type", 
    target_col = "fine_dust(㎍/㎥)")

# 결과를 딕셔너리로 정리
double_group_stats = {
    "Year-Grade": df_year_grade_stats,
    "Year-Season": df_year_season_stats,
    "Year-Day": df_year_day_stats,
    "Grade-Season": df_grade_season_stats,
    "Grade-Day": df_grade_day_stats,
    "Season-Day": df_season_day_stats
}

# 결과 출력
print("2. 이중 그룹 분석")
for key, value in double_group_stats.items():
    print(f"\n{key} Stats:\n", value)

2. 이중 그룹 분석

Year-Grade Stats:
                  Max    Min        Mean      Variance  Count  Median      Q1  \
year grade                                                                     
2021 good       30.0    9.0   21.496063     26.744032    127    22.0   18.00   
     normal     78.0   31.0   48.407407    161.912924    189    46.0   38.00   
     bad       146.0   81.0  107.558824    291.041889     34   107.5   96.00   
     very_bad  389.0  156.0  262.500000  11896.300000      6   243.5  166.50   
2022 good       30.0    8.0   22.981481     27.819531    162    24.0   19.25   
     normal     79.0   31.0   44.110465    129.853223    172    41.0   34.75   
     bad       129.0   88.0  104.333333    205.466667      6   100.5   97.00   
     very_bad  247.0  169.0  208.000000   3042.000000      2   208.0  188.50   
2023 good       30.0   10.0   21.866197     30.201828    142    22.0   18.25   
     normal     78.0   31.0   46.273224    176.309554    183    42.0   35.00   
     bad

- 두 개 변수 조합 분석 시각화

3. 세 개 변수 조합 분석

In [ ]:
# 3-1. 연도와 등급과 계절을 조합해서 분석
# 3-2. 연도와 등급과 평일/주말을 조합해서 분석
# 3-3. 등급과 계절과 평일/주말을 주합해서 분석

In [19]:
def group_analysis_by_three_columns(df, group_col1, group_col2, group_col3, target_col):
    # 세 컬럼으로 그룹화
    grouped_df = df.groupby([group_col1, group_col2, group_col3], observed=True)

    # 통계 계산 및 컬럼 이름 지정
    stats = grouped_df[target_col].agg([
    ("Max", "max"),                    # (열 이름, 계산 방법)
    ("Min", "min"),
    ("Mean", "mean"),
    ("Variance", "var"),
    ("Count", "count"),
    ("Median", "median"),                   # 중앙값
    ("Q1", lambda x: x.quantile(0.25)),     # 1사분위수 (Q1)
    ("Q3", lambda x: x.quantile(0.75)),     # 3사분위수 (Q3)
    ("IQR", lambda x: x.quantile(0.75) - x.quantile(0.25)),     # IQR
    ("Skewness", lambda x: skew(x)),        # 왜도
    ("Kurtosis", lambda x: kurtosis(x)),    # 첨도
    ("Range", lambda x: x.max() - x.min())  # 범위
])
    return stats

# 함수 실행 예시
df_year_grade_season_stats = group_analysis_by_three_columns(
                                df = m_fine_dust,
                                group_col1 = "year", 
                                group_col2 = "grade", 
                                group_col3 = "season", 
                                target_col = "fine_dust(㎍/㎥)")
df_year_grade_day_stats = group_analysis_by_three_columns(
                                df = m_fine_dust,
                                group_col1 = "year", 
                                group_col2 = "grade", 
                                group_col3 = "day_type", 
                                target_col = "fine_dust(㎍/㎥)")
df_grade_season_day_stats = group_analysis_by_three_columns(
                                df = m_fine_dust,
                                group_col1 = "grade", 
                                group_col2 = "season", 
                                group_col3 = "day_type", 
                                target_col = "fine_dust(㎍/㎥)")

# 결과를 딕셔너리로 정리
triple_group_stats = {
    "Year-Grade-Season": df_year_grade_season_stats,
    "Year-Grade-Day": df_year_grade_day_stats,
    "Grade-Season-Day": df_grade_season_day_stats,
}

# 결과 출력
print("2. 삼중 그룹 분석")
for key, value in triple_group_stats.items():
    print(f"\n{key} Stats:\n", value)

2. 삼중 그룹 분석

Year-Grade-Season Stats:
                         Max    Min        Mean      Variance  Count  Median  \
year grade    season                                                          
2021 good     Spring   30.0   14.0   23.071429     22.686813     14    24.0   
              Summer   30.0   15.0   23.173913     18.546860     46    23.0   
              Fall     30.0    9.0   18.705882     26.651765     51    18.0   
              Winter   30.0   19.0   24.187500     15.362500     16    24.0   
     normal   Spring   78.0   31.0   51.900000    143.969388     50    51.5   
              Summer   72.0   32.0   44.652174    104.276329     46    42.0   
              Fall     74.0   31.0   44.571429    143.134454     35    40.0   
              Winter   78.0   31.0   50.689655    206.147610     58    49.5   
     bad      Spring  132.0   81.0  106.705882    296.970588     17   107.0   
              Fall    140.0  102.0  121.000000    722.000000      2   121.0   
             

- 세 개 변수 조합 분석 시각화

4. 모든 변수 조합 분석

In [ ]:
# 4-1. 연도와 등급과 계절과 평일/주말을 조합해서 분석

In [23]:
def group_analysis_by_four_columns(df, group_col1, group_col2, group_col3, group_col4, target_col):
    # 네 컬럼으로 그룹화
    grouped_df = df.groupby([group_col1, group_col2, group_col3, group_col4], observed=True)

    # 통계 계산 및 컬럼 이름 지정
    stats = grouped_df[target_col].agg([
    ("Max", "max"),                    # (열 이름, 계산 방법)
    ("Min", "min"),
    ("Mean", "mean"),
    ("Variance", "var"),
    ("Count", "count"),
    ("Median", "median"),                   # 중앙값
    ("Q1", lambda x: x.quantile(0.25)),     # 1사분위수 (Q1)
    ("Q3", lambda x: x.quantile(0.75)),     # 3사분위수 (Q3)
    ("IQR", lambda x: x.quantile(0.75) - x.quantile(0.25)),     # IQR
    ("Skewness", lambda x: skew(x)),        # 왜도
    ("Kurtosis", lambda x: kurtosis(x)),    # 첨도
    ("Range", lambda x: x.max() - x.min())  # 범위
])
    return stats

# 함수 실행 예시
df_year_season_grade_day_stats = group_analysis_by_four_columns(
                                df = m_fine_dust,
                                group_col1 = "year", 
                                group_col2 = "season",
                                group_col3 = "grade",  
                                group_col4 = "day_type",
                                target_col = "fine_dust(㎍/㎥)")

# 결과를 딕셔너리로 정리
quadruple_group_stats = {
    "Year-Grade-Season": df_year_season_grade_day_stats
}

# 결과 출력
print("2. 사중 그룹 분석")
for key, value in quadruple_group_stats.items():
    print(f"\n{key} Stats:\n", value)

2. 사중 그룹 분석

Year-Grade-Season Stats:
                                Max   Min        Mean    Variance  Count  \
year season grade  day_type                                               
2021 Spring good   weekday    30.0  16.0   24.000000   19.777778     10   
                   weekend    26.0  14.0   20.750000   28.916667      4   
            normal weekday    78.0  35.0   54.181818  136.278409     33   
                   weekend    74.0  31.0   47.470588  136.764706     17   
            bad    weekday   132.0  81.0  106.071429  356.686813     14   
...                            ...   ...         ...         ...    ...   
2024 Winter good   weekday    30.0  11.0   21.666667   33.666667     15   
                   weekend    28.0  14.0   22.727273   23.618182     11   
            normal weekday    77.0  31.0   45.863636  176.123377     22   
                   weekend    69.0  31.0   45.857143  227.142857      7   
            bad    weekday    83.0  83.0   83.000000         

딕셔너리를 csv파일로 저장하기

In [ ]:
def save_stats_with_check(combined_stats, double_group_stats, triple_group_stats, quadruple_group_stats):
    print("데이터 구조 확인 중...")
    print_data_structure(combined_stats, "combined_stats")
    print_data_structure(double_group_stats, "double_group_stats")
    print_data_structure(triple_group_stats, "triple_group_stats")
    print_data_structure(quadruple_group_stats, "quadruple_group_stats")
    
    print("\n저장 시작...")
    save_multiindex_stats_to_csv(
        combined_stats,
        double_group_stats,
        triple_group_stats,
        quadruple_group_stats
    )

def print_data_structure(data, name):
    """데이터 구조를 출력하여 확인"""
    print(f"{name} 구조:")
    if isinstance(data, dict):
        print({key: type(value) for key, value in data.items()})
    else:
        print(f"{name} is {type(data)}")

def save_multiindex_stats_to_csv(*args):
    """각 딕셔너리 내 DataFrame을 개별 CSV로 저장"""
    for i, stats in enumerate(args, start=1):
        if isinstance(stats, dict):
            for key, df in stats.items():
                if isinstance(df, pd.DataFrame):
                    filename = f"stats_{i}_{key}.csv"
                    df.to_csv(filename)
                    print(f"{filename} 파일로 저장되었습니다.")
                else:
                    print(f"{key}는 DataFrame 형식이 아니므로 CSV로 저장할 수 없습니다.")
        else:
            print(f"stats_{i}는 딕셔너리 형식이 아닙니다.")

# 예제 실행 (실제 데이터로 교체)
save_stats_with_check(combined_stats, double_group_stats, triple_group_stats, quadruple_group_stats)


데이터 구조 확인 중...
combined_stats 구조:
{'Year': <class 'pandas.core.frame.DataFrame'>, 'Grade': <class 'pandas.core.frame.DataFrame'>, 'Season': <class 'pandas.core.frame.DataFrame'>, 'Day Type': <class 'pandas.core.frame.DataFrame'>}
double_group_stats 구조:
{'Year-Grade': <class 'pandas.core.frame.DataFrame'>, 'Year-Season': <class 'pandas.core.frame.DataFrame'>, 'Year-Day': <class 'pandas.core.frame.DataFrame'>, 'Grade-Season': <class 'pandas.core.frame.DataFrame'>, 'Grade-Day': <class 'pandas.core.frame.DataFrame'>, 'Season-Day': <class 'pandas.core.frame.DataFrame'>}
triple_group_stats 구조:
{'Year-Grade-Season': <class 'pandas.core.frame.DataFrame'>, 'Year-Grade-Day': <class 'pandas.core.frame.DataFrame'>, 'Grade-Season-Day': <class 'pandas.core.frame.DataFrame'>}
quadruple_group_stats 구조:
{'Year-Grade-Season': <class 'pandas.core.frame.DataFrame'>}

저장 시작...
stats_1_Year.csv 파일로 저장되었습니다.
stats_1_Grade.csv 파일로 저장되었습니다.
stats_1_Season.csv 파일로 저장되었습니다.
stats_1_Day Type.csv 파일로 저장되었습니다.
stat

- 모든 변수 조합 분석 시각화

In [ ]:
# 그래프 그리기
count_data.plot(kind='bar', stacked=False, color=['skyblue', 'salmon'])
plt.title("Fine Dust Level by Weekday and Grade")
plt.xlabel("Grade")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
# 그래프 그리기
fig, axes = plt.subplots(len(grouped_data.index.levels[0]), 1, figsize=(10, 10), sharex=True)
fig.suptitle("Fine Dust Level by Year, Season, Weekday/Weekend, and Grade")

# 색상 정의 (순서대로 good, normal, bad, very_bad에 대응)
colors = ['lightgreen', 'skyblue', 'salmon', 'orange']

# 각 연도별로 서브플롯에 그래프 생성
for i, (year, data) in enumerate(grouped_data.groupby(level=0)):
    data = data.droplevel(0)  # 연도 레벨 제거
    
    # season_order에 따라 계절 순서 정렬
    data = data.loc[season_order.categories]
    
    # 정의된 순서대로 그래프 그리기
    ax = data.plot(kind='bar', stacked=False, ax=axes[i], color=colors)
    axes[i].set_title(f"{year}년")
    axes[i].set_ylabel("Count")
    
    # x축 레이블 회전 제거
    axes[i].tick_params(axis='x', rotation=0)
    
    # 범례 순서 명시적 지정
    handles, labels = axes[i].get_legend_handles_labels()
    axes[i].legend(handles, labels, bbox_to_anchor=(1, 0.8))

plt.xlabel("Season and Grade")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()